## REBEL: RErank BEyond reLevance
This notebook will walk you through how to set up and run the REBEL method for RAG reranking

In [2]:
import os

OPENAI_API_KEY="sk-" # put your openai key here
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
OPENAI_API_BASE="" # optional, if you want to use another hosted endpoint
if OPENAI_API_BASE:
    os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE


In [3]:
# pip install llama_index from our fork, once the PR is merged this will just be a normal llama_index install
!git clone https://github.com/bvarjavand/llama_index.git
!cd llama_index && pip install --quiet -e .
!cd llama_index/llama-index-core && pip install --quiet -e .

fatal: destination path 'llama_index' already exists and is not an empty directory.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llama-index (pyproject.toml) ... done
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llama-index-core (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-agent-openai 0.4.3 requires llama-index-core<0.13.0,>=0.12.11, but you have llama-index-core 0.12.10 which is incompatible.


In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.postprocessor import LLMRerank
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display
from llama_index.core import Settings
import nest_asyncio
nest_asyncio.apply()

if OPENAI_API_BASE:
    Settings.llm = OpenAI(temperature=0, api_key=OPENAI_API_KEY, model="gpt-4o", api_base=OPENAI_API_BASE)
else:
    Settings.llm = OpenAI(temperature=0, api_key=OPENAI_API_KEY, model="gpt-4o")
Settings.chunk_size = 512

## Build RAG index

In [5]:
from pathlib import Path
import requests
from llama_index.embeddings.openai import OpenAIEmbedding

# save documents from wikipedia with these title(s):
wiki_titles = [
    "Vincent van Gogh",
]

# pull documents from wikipedia
data_path = Path("data_wiki")
for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

if OPENAI_API_BASE:
    embed_model = OpenAIEmbedding(model="text-embedding-3-large", api_key=OPENAI_API_KEY, api_base=OPENAI_API_BASE)
else:
    embed_model = OpenAIEmbedding(model="text-embedding-3-large", api_key=OPENAI_API_KEY)

# load documents
documents = SimpleDirectoryReader("./data_wiki/").load_data()
# build index
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

## Define query string, retrieve nodes, and rerank them

In [7]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.core.postprocessor import REBELRerank

### You can define any query string you want here ###
query_str = "Which date did Paul Gauguin arrive in Arles?"
query_bundle = QueryBundle(query_str)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=50,
)

# retrieve nodes
retrieved_nodes = retriever.retrieve(query_bundle)

# configure reranker
if OPENAI_API_BASE:
    reranked = REBELRerank(llm=OpenAI(model='gpt-4o', api_base=OPENAI_API_BASE), top_n=2)
else:
    reranked = REBELRerank(llm=OpenAI(model='gpt-4o'), top_n=2)

# rerank nodes
reranked_nodes = reranked.postprocess_nodes(retrieved_nodes, query_bundle)

In [8]:
import pandas as pd
import torch
from IPython.display import display, HTML

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

### Top 3 nodes from initial retrieval:

In [9]:
visualize_nodes(retrieved_nodes[:2])

,Score,Text
0,0.514267,"Discussions on art, artists, and their social situations started during this exhibition, continued and expanded to include visitors to the show, like Camille Pissarro and his son Lucien, Signac and Seurat. In February 1888, feeling worn out from life in Paris, Van Gogh left, having painted more than 200 paintings during his two years there. Hours before his departure, accompanied by Theo, he paid his only visit to Seurat in his studio.=== Artistic breakthrough ======= Arles (1888–89) ====Ill from drink and suffering from smoker's cough, in February 1888, Van Gogh sought refuge in Arles. He seems to have moved with thoughts of founding an art colony. The Danish artist Christian Mourier-Petersen was his companion for two months and at first, Arles appeared exotic to Van Gogh. In a letter, he described it as a foreign country: ""The Zouaves, the brothels, the adorable little Arlésienne going to her First Communion, the priest in his surplice, who looks like a dangerous rhinoceros, the people drinking absinthe, all seem to me creatures from another world.""The time in Arles was one of Van Gogh's more prolific periods: he completed 200 paintings and more than 100 drawings and watercolours. He was energised by the local countryside and light; his works from this period are rich in yellow, ultramarine and mauve. They include harvests, wheat fields and general rural landmarks from the area, including The Old Mill (1888), one of seven canvases sent to Pont-Aven on 4 October 1888 in an exchange of works with Paul Gauguin, Émile Bernard, Charles Laval and others. In March 1888, Van Gogh created landscapes using a gridded ""perspective frame"" and three of those works were shown at the annual exhibition of the Société des Artistes Indépendants. In April, he was visited by the American artist Dodge MacKnight, who was living nearby at Fontvieille. On 1 May 1888, Van Gogh signed a lease for four rooms at 2 Place Lamartine, Arles, which he later painted in The Yellow House."
1,0.507679,"Van Gogh wrote that with The Night Café he tried ""to express the idea that the café is a place where one can ruin oneself, go mad, or commit a crime"". When he visited Saintes-Maries-de-la-Mer in June, he gave lessons to a Zouave second lieutenant – Paul-Eugène Milliet – and painted boats on the sea and the village. MacKnight introduced Van Gogh to Eugène Boch, a Belgian painter who sometimes stayed in Fontvieille, and the two exchanged visits in July.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realise his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers."" When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House. When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les

### Top 3 nodes from reranking

In [10]:
visualize_nodes(reranked_nodes[:2])

,Score,Text
0,1.0,"Van Gogh wrote that with The Night Café he tried ""to express the idea that the café is a place where one can ruin oneself, go mad, or commit a crime"". When he visited Saintes-Maries-de-la-Mer in June, he gave lessons to a Zouave second lieutenant – Paul-Eugène Milliet – and painted boats on the sea and the village. MacKnight introduced Van Gogh to Eugène Boch, a Belgian painter who sometimes stayed in Fontvieille, and the two exchanged visits in July.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realise his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers."" When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House. When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh.Van Gogh and Gauguin visited Montpellier in December 1888, where they saw works by Courbet and Delacroix in the Musée Fabre. Their relationship began to deteriorate; Van Gogh admired Gauguin and wanted to be treated as his equal, but Gauguin was arrogant and domineering, which frustrated Van Gogh. They often quarrelled; Van Gogh increasingly feared that Gauguin was going to desert him, and the situation, which Van Gogh described as one of ""excessive tension"", rapidly headed towards crisis point."
1,1.0,"A woman must have hips, buttocks, a pelvis in which she can carry a baby!' According to some accounts, this was the last time Van Gogh attended classes at the academy and he left later for Paris. On 31 March 1886, which was about a month after the confrontation with Siberdt, the teachers of the academy decided that 17 students, including Van Gogh, had to repeat a year. The story that Van Gogh was expelled from the academy by Siberdt is therefore unfounded.==== Paris (1886–1888) ====Van Gogh moved to Paris in March 1886 where he shared Theo's rue Laval apartment in Montmartre and studied at Fernand Cormon's studio. In June the brothers took a larger flat at 54 rue Lepic. In Paris, Vincent painted portraits of friends and acquaintances, still life paintings, views of Le Moulin de la Galette, scenes in Montmartre, Asnières and along the Seine. In 1885 in Antwerp he had become interested in Japanese ukiyo-e woodblock prints and had used them to decorate the walls of his studio; while in Paris he collected hundreds of them. He tried his hand at Japonaiserie, tracing a figure from a reproduction on the cover of the magazine Paris Illustre, The Courtesan or Oiran (1887), after Keisai Eisen, which he then graphically enlarged in a painting.After seeing the portrait of Adolphe Monticelli at the Galerie Delareybarette, Van Gogh adopted a brighter palette and a bolder attack, particularly in 